# Bias Article Responses - Refactored Notebook

In [1]:

import json
import pandas as pd
from llm import CustomLLM
    

## Function to Load JSON Data

In [2]:

def load_json_data(filepath):
    """Loads JSON data from the provided file path."""
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: File {filepath} not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Failed to decode JSON from {filepath}.")
        return None
    

In [3]:
def load_question_code_mapping(question_codes_path):
    """Load the question-to-code mapping from the question_codes.json file."""
    question_codes_data = load_json_data(question_codes_path)
    
    valid_question_codes = ['F1A10_1', 'F2A6', 'F2A7', 'F2A9', 'F3A3_1', 'F3A6_1', 'F3A7_1', 'F3A8_1']
    
    # Filter out questions with invalid codes
    filtered_data = [entry for entry in question_codes_data if entry['code'] in valid_question_codes]

    question_code_mapping = {}
    for entry in filtered_data:
        question = entry['question']
        code = entry['code']
        options = entry['options']
        question_code_mapping[question] = {'code': code, 'options': options}

    return question_code_mapping

## Function to Generate Prompts

In [4]:

def generate_prompt(question, article, options):
    """Generates a prompt from a question, article, and list of options."""
    article_source = article['source']
    article_text = article['text']
    article_bias = article['bias']  # Capture bias
    
    # Format the options as a readable string
    options_text = "\n".join([f"- {option}" for option in options.values()])
    
    # Create the prompt
    prompt_text = (
    f"You have stumbled upon the following news article, so you read it:\n\n"
    f"\"{article_text}\"\n\n"
    f"Reflecting on the emotions, perceptions, and intuitions the article triggered, and still based on your background and beliefs, "
    f"answer the following question by selecting only **ONE** of the provided options. "
    f"Your response must contain only the selected option, exactly as written. No explanation or additional text is allowed.\n\n"
    f"Any response that does not use exactly one option will be considered invalid.\n\n"
    f"**Question**: {question}\n"
    f"**Options** (choose one):\n{options_text}"
)
    
    return {
        "prompt": prompt_text,
        "bias": article_bias,
        "question": question,
        "article_source": article_source
    }

## Function to Generate Prompts with Bias from Articles

In [5]:
def generate_prompts_with_bias(question_codes_path, articles_bias_path, output_path='../data/processed/prompts_with_articles.json'):
    """Generates prompts for each valid question based on its biased articles and options, and stores the result in a JSON file."""

    # Load the question codes and articles JSON files
    question_data = load_question_code_mapping(question_codes_path)
    article_data = load_json_data(articles_bias_path)

    prompts = []

    # Loop over each question in the JSON data
    for question, item in question_data.items():  # 'question' is the question string, 'item' is the dictionary holding the code and options
        question_code = item.get('code')  # Get the code from the item's dictionary
        options = item.get('options')  # Get the options from the item's dictionary

        # Check if the question has a valid code
        if question_code is None:
            print(f"Skipping question '{question}' with missing code")
            continue

        print(f"Processing question '{question}' with code '{question_code}'")

        # Find the corresponding article using the question code
        corresponding_article = next(
            (a for a in article_data if a.get('question', {}).get('code') == question_code),
            None
        )

        if corresponding_article:
            articles = corresponding_article.get('articles_found', [])

            # For each article, generate a prompt using the generate_prompt function
            for article in articles:
                # Modify prompt_entry to include the question code
                prompt_entry = {
                    "question": question,
                    "question_code": question_code,  # Include the question code
                    "article": article,
                    "options": options
                }
                prompts.append(prompt_entry)
        else:
            print(f"No articles found for question: '{question}'")

    # Save the generated prompts to a file
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(prompts, f, indent=4)

    print(f"Prompts have been successfully saved to {output_path}")

    return prompts


## Function to Process LLM Responses with Checkpoints

In [6]:
def process_responses_with_checkpoints(llm, parser, prompts_file, processed_responses, output_file):
    """
    Processes LLM responses for users based on their saved question prompts and writes the results to a file.
    Saves checkpoints after processing each user to avoid losing data in case of a connection drop.

    Parameters:
    - llm: An LLM object that provides a method 'generate_response()'.
    - parser: A parser object to process and parse LLM responses.
    - prompts_file (str): Path to the saved prompts JSON file.
    - processed_responses (dict): Dictionary tracking which user-article pairs have been processed.
    - output_file (str): Path where the processed responses will be written.
    """
    
    user_responses = []

    # Load existing responses from the file, if any, to avoid duplication
    try:
        with open(output_file, 'r', encoding='utf-8') as f:
            for line in f:
                user_responses.append(json.loads(line.strip()))
    except FileNotFoundError:
        print(f"File {output_file} not found. Creating a new file.")
    except json.JSONDecodeError:
        print("Error reading existing responses. Starting fresh.")

    # Load the saved prompts from the file using load_json_data
    question_prompts_with_bias = load_json_data(prompts_file)
    if question_prompts_with_bias is None:
        print(f"Failed to load prompts from {prompts_file}. Exiting.")
        return

    print(f"Loaded {len(question_prompts_with_bias)} prompts from {prompts_file}.")

    # Loop through each user's responses
    for user_id, persona_prompt in processed_responses.items():
        for question_entry in question_prompts_with_bias:
            question_prompt = question_entry["question"]  # Get the question text
            question_code = question_entry["question_code"]  # Get the question code
            article = question_entry["article"]  # Single article per entry
            
            # Process each article for the question
            article_source = article.get("source")

            # Skip already processed pairs
            if question_prompt in processed_responses[user_id] and article_source in processed_responses[user_id][question_prompt]:
                print(f"Skipping already processed User ID: {user_id}, Question: {question_prompt}, Article Source: {article_source}")
                continue

            # Get the full prompt directly from the saved data (generate using question and article)
            full_prompt = generate_prompt(question_prompt, article, question_entry['options'])["prompt"]

            # Send persona prompt and question prompt to LLM
            response = llm.generate_response(persona_prompt, full_prompt)
            parsed_output = parser.parse(response)
            
            # Collect the responses for the current user
            user_responses.append({
                "user_id": user_id,
                "question": question_prompt,
                "question_code": question_code,  # Include question code in the response
                "article_source": article_source,  # Track the source of the article for uniqueness
                "bias": article.get("bias", "Unknown"),  # Track the bias of the article
                "response": parsed_output
            })

            # Mark this question-article as processed for this user
            if question_prompt not in processed_responses[user_id]:
                processed_responses[user_id][question_prompt] = set()
            processed_responses[user_id][question_prompt].add(article_source)

            # Write the response incrementally (checkpoint after each article)
            try:
                with open(output_file, 'a', encoding='utf-8') as f:
                    f.write(json.dumps(user_responses[-1]) + '\n')  # Write the latest response
            except IOError:
                print(f"Error: Could not write to file {output_file}")

    print(f"Processing completed. All responses saved to {output_file}")


#### Output Parser

In [7]:
import logging
import re

class CustomOutputParser:
    def __init__(self):
        # Combined list of all predefined options, sorted by length (longest first to avoid partial matches)
        self.target_keys = sorted([
            # Agreement options
            'Completely agree', 'Agree', 'Agree to some extent', 'Neither agree nor disagree',
            'Disagree to some extent', 'Disagree', 'Completely disagree',
            # Concern levels
            'not concerned at all', 'not very concerned', 'moderately concerned',
            'quite concerned', 'very concerned'
        ], key=len, reverse=True)  # Sort by length to prioritize longer options

    def parse(self, output: str) -> dict:
        try:
            # Attempt to extract the exact option
            selected_option = self.extract_option(output)
            logging.debug(f"Extracted option: {selected_option}")

            # Handle case when no valid option is found
            if selected_option is None:
                logging.warning("No valid option found in the output.")
                selected_option = "N/A"

            return {
                "type": "final_answer",
                "selected_option": selected_option.strip()
            }
        except Exception as e:
            logging.error(f"Error parsing LLM output: {e}")
            return {"type": "raw_output", "content": output.strip()}

    def extract_option(self, text: str) -> str:
        # Normalize the text (remove extra spaces, lowercase for comparison)
        normalized_text = text.strip().lower()

        # Check for exact matches of predefined options in the raw output
        for option in self.target_keys:
            # Use word boundaries to ensure we're matching the whole phrase
            pattern = re.compile(r'\b' + re.escape(option.lower()) + r'\b')
            if pattern.search(normalized_text):
                return option  # Return the exact matched option

        # No valid option found
        return None

# Ensure logging is set up to show debug information
logging.basicConfig(level=logging.DEBUG)


## Performing the Code Flow

In [8]:
def convert_sets_to_lists(obj):
    """Recursively converts sets to lists in a nested dictionary."""
    if isinstance(obj, dict):
        return {k: convert_sets_to_lists(v) for k, v in obj.items()}
    elif isinstance(obj, set):
        return list(obj)
    return obj

In [9]:
def main(question_codes_path, articles_bias_path, processed_responses_path, persona_prompts_path, output_file, llm, parser):
    """
    Main function to generate prompts with bias and process LLM responses, with persona integration.
    
    Parameters:
    - question_codes_path (str): Path to the question codes JSON file.
    - articles_bias_path (str): Path to the articles bias JSON file.
    - processed_responses_path (str): Path to the processed responses JSON file (checkpoint tracking).
    - persona_prompts_path (str): Path to the persona prompts JSON file.
    - output_file (str): Path where LLM responses will be written.
    - llm: An LLM object that provides a method 'generate_response()'.
    - parser: A CustomOutputParser object to process and parse LLM responses.
    """

    # Step 1: Load persona prompts
    print("Loading persona prompts...")
    persona_prompts = load_json_data(persona_prompts_path)
    if not persona_prompts:
        print(f"Error: Could not load persona prompts from {persona_prompts_path}.")
        return
    print(f"Loaded {len(persona_prompts)} persona prompts.")
    print(f"Sample persona prompt: {persona_prompts[0]}")  # Debug: Print first persona prompt to check structure

    # Step 2: Load question-code mapping and generate prompts with bias
    print("Loading question-code mapping and generating prompts with bias...")
    question_code_mapping = load_question_code_mapping(question_codes_path)
    print(f"Loaded question-code mapping for {len(question_code_mapping)} questions.")
    print(f"Sample question-code mapping: {list(question_code_mapping.items())[0]}")  # Debug: Print first entry

    prompts_with_bias = generate_prompts_with_bias(question_codes_path, articles_bias_path)
    if not prompts_with_bias:
        print(f"Error: Could not generate prompts with bias from {question_codes_path} and {articles_bias_path}.")
        return
    print(f"Generated {len(prompts_with_bias)} prompts with bias.")
    print(f"Sample prompt with bias: {prompts_with_bias[0]}")  # Debug: Print first prompt to check structure

    # Step 3: Load or initialize processed responses
    try:
        print(f"Loading processed responses from {processed_responses_path}...")
        with open(processed_responses_path, 'r', encoding='utf-8') as f:
            processed_responses = json.load(f)
        print(f"Loaded processed responses for {len(processed_responses)} users.")
    except FileNotFoundError:
        print(f"File {processed_responses_path} not found. Creating an empty responses file.")
        processed_responses = {}
    except json.JSONDecodeError:
        print(f"Error: {processed_responses_path} is either empty or corrupted. Initializing an empty processed responses structure.")
        processed_responses = {}

    print(f"Sample processed response: {list(processed_responses.items())[0] if processed_responses else 'No processed responses yet.'}")  # Debug: Print sample processed response

    # Step 4: Process LLM responses
    print("Processing LLM responses...")
    
    for persona in persona_prompts:
        persona_prompt = persona["persona_prompt"]
        user_id = persona["user_id"]

        print(f"Processing for User ID: {user_id}...")

        # Ensure user's responses are initialized in the processed_responses structure
        if user_id not in processed_responses:
            processed_responses[user_id] = {}

        user_responses = []  # For collecting responses to write for this user

        for prompt in prompts_with_bias:
            question = prompt.get('question', 'Unknown Question').strip()  # Normalize the question
            question_code = prompt.get('question_code')

            # Use question-code mapping if necessary
            if question_code is None:
                question_code = question_code_mapping.get(question)

            if question_code is None:
                print(f"Warning: No question_code for question: {question}")
                continue

            print(f"Processing question: {question}, Code: {question_code}")  # Debug: See which question is being processed

            articles = [prompt['article']] if isinstance(prompt['article'], dict) else prompt['article']  # Handle single articles or lists
            print(f"Found {len(articles)} articles for question: {question}")  # Debug: See how many articles are found

            # Process all articles for the question
            for article_index, article in enumerate(articles):
                article_source = article.get("source")
                print(f"Processing article {article_index + 1}/{len(articles)} for question: {question}, Source: {article_source}, Bias: {article.get('bias', 'Unknown')}")  # Debug: Print article source and bias

                # Skip already processed pairs
                if question in processed_responses[user_id] and article_source in processed_responses[user_id][question]:
                    print(f"Skipping already processed User ID {user_id}, Question: {question}, Article Source: {article_source}")
                    continue

                # Generate the full prompt using the article and options
                prompt_info = generate_prompt(question, article, prompt['options'])
                full_prompt = prompt_info['prompt']

                # Send persona prompt and question prompt to LLM
                llm_response = llm.generate_response(persona_prompt, full_prompt)
                if not llm_response:
                    print(f"LLM returned an empty response for User ID {user_id}, Question: {question}")
                    continue

                # Parse the response using the parser
                parsed_response = parser.parse(llm_response)
                if not parsed_response:
                    print(f"Parsing returned an empty response for User ID {user_id}, Question: {question}")
                    continue

                # Save the response in the processed_responses dictionary
                if question not in processed_responses[user_id]:
                    processed_responses[user_id][question] = set()
                processed_responses[user_id][question].add(article_source)

                # Collect the responses for writing
                user_responses.append({
                    "user_id": user_id,
                    "question": question,
                    "question_code": question_code,  # Include question code in the response
                    "article_index": article_index,
                    "bias": article.get('bias', 'Unknown'),
                    "response": parsed_response
                })

        # Step 5: Save processed responses incrementally after each user
        processed_responses_serializable = convert_sets_to_lists(processed_responses)
        with open(processed_responses_path, 'w', encoding='utf-8') as f:
            json.dump(processed_responses_serializable, f, indent=4)

        # Step 6: Save final LLM responses for this user incrementally after processing the user
        with open(output_file, 'a', encoding='utf-8') as f:
            json.dump(user_responses, f, indent=4)

    print("Done.")


In [10]:
main(
    persona_prompts_path='../data/processed/persona_prompts.json',
    question_codes_path='../data/raw/question_codes.json',
    articles_bias_path='../data/raw/articles_bias.json',
    processed_responses_path='../data/processed/processed_responses.json',
    output_file='../data/processed/after_responses.json',
    parser=CustomOutputParser(),
    llm=CustomLLM(model="llama3.1:70b-instruct-q6_K", api_url="https://inf.cl.uni-trier.de/")
)


2024-09-10 14:43:16,604 - DEBUG - Starting new HTTPS connection (1): inf.cl.uni-trier.de:443


Loading persona prompts...
Loaded 117 persona prompts.
Sample persona prompt: {'user_id': 'IDUS103408', 'persona_prompt': "You are a social media user, and the following information represents your political stance and opinions. Use the given political context to guide your responses and actions:\n\n1. **Democratic Party**: Your stance is that the Democratic Party is ranked at 'Extreme Right' on the political scale.\n2. **Republican Party**: Your stance is that the Republican Party is ranked at 'Extreme Left' on the political scale.\n3. **Your Own Political Position**: You consider your political position to be 'Extreme Right' on the political scale.\n4. **2020 Presidential Election Vote**: Your voting information for the 2020 presidential election is 'Joe Biden'.\n5. **2024 Presidential Election Vote**: Your hypothetical voting choice for the 2024 presidential election is 'Joe Biden'.\n\n**Political Context**: {'Core Beliefs and Values': ['Nationalism: Emphasizing national sovereignty

2024-09-10 14:43:18,262 - DEBUG - https://inf.cl.uni-trier.de:443 "POST / HTTP/11" 200 6274
2024-09-10 14:43:18,265 - DEBUG - Extracted option: Disagree to some extent
2024-09-10 14:43:18,269 - DEBUG - Starting new HTTPS connection (1): inf.cl.uni-trier.de:443


Processing question: I feel like I am treated fairly by politicians., Code: F1A10_1
Found 1 articles for question: I feel like I am treated fairly by politicians.
Processing article 1/1 for question: I feel like I am treated fairly by politicians., Source: Fox News, Bias: right


2024-09-10 14:43:19,166 - DEBUG - https://inf.cl.uni-trier.de:443 "POST / HTTP/11" 200 5479
2024-09-10 14:43:19,169 - DEBUG - Extracted option: Completely agree
2024-09-10 14:43:19,169 - DEBUG - Starting new HTTPS connection (1): inf.cl.uni-trier.de:443


...

Processing question: Agree or disagree: Foreigners are taking jobs away from Americans, Code: F3A8_1
Found 1 articles for question: Agree or disagree: Foreigners are taking jobs away from Americans
Processing article 1/1 for question: Agree or disagree: Foreigners are taking jobs away from Americans, Source: MSNBC, Bias: left


2024-09-10 15:25:49,956 - DEBUG - https://inf.cl.uni-trier.de:443 "POST / HTTP/11" 200 7659
2024-09-10 15:25:49,957 - DEBUG - Extracted option: Completely disagree
2024-09-10 15:25:49,960 - DEBUG - Starting new HTTPS connection (1): inf.cl.uni-trier.de:443


Processing question: Agree or disagree: Foreigners are taking jobs away from Americans, Code: F3A8_1
Found 1 articles for question: Agree or disagree: Foreigners are taking jobs away from Americans
Processing article 1/1 for question: Agree or disagree: Foreigners are taking jobs away from Americans, Source: Fox News, Bias: right


2024-09-10 15:25:51,241 - DEBUG - https://inf.cl.uni-trier.de:443 "POST / HTTP/11" 200 6604
2024-09-10 15:25:51,241 - DEBUG - Extracted option: Disagree to some extent


Done.
